In [123]:
import pandas as pd
import numpy as np
from globals import BASE_DIR
import os

available_datasets = []

In [124]:
available_datasets = ["brightkite", "foursquaretky", "gowalla", "snowcard", "yelp"]

In [125]:


# Function to format the best and second-best values
def format_best_second(row, col_group, best_criteria):
    """Formatting the best row to be bold and the second one to be underlined"""
    values = row[col_group]
    
    if best_criteria == "highest":  # For ndcg
        sorted_indices = np.argsort(values)[::-1]  # Descending
    elif best_criteria == "lowest":  # For arp
        sorted_indices = np.argsort(values)  # Ascending
    elif best_criteria == "closest_to_zero":  # For poplift
        sorted_indices = np.argsort(np.abs(values))  # Closest to zero

    # Mark best (bold) and second-best (underline)
    formatted = values.astype(str)  # Convert to strings for LaTeX formatting
    if len(sorted_indices) > 0:
        formatted[sorted_indices[0]] = f"\\textbf{{{formatted[sorted_indices[0]]}}}"  # Bold
    if len(sorted_indices) > 1:
        formatted[sorted_indices[1]] = f"\\underline{{{formatted[sorted_indices[1]]}}}"  # Underline

    return formatted







In [126]:
for dataset in available_datasets:
    df = pd.read_csv(f"{BASE_DIR}/{dataset}_dataset/evaluation_results.csv")
    
    df["user_group"] = df["user_group"].apply(lambda x: x.replace("high", "g1").replace("medium", "g2").replace("low", "g3"))

    float_columns = df.select_dtypes(include="float").columns
    df[float_columns] = df[float_columns].round(4)


    df_pivot = df.pivot(
    index=["model", "user_group"],
    columns="method",
    values=["ndcg", "arp", "poplift"]
    )

    # Apply formatting for each metric group
    metric_groups = ["ndcg", "arp", "poplift"]
    criteria = {"ndcg": "highest", "arp": "lowest", "poplift": "closest_to_zero"}

    for metric in metric_groups:
        method_cols = [col for col in df_pivot.columns if col[0] == metric]
        df_pivot[method_cols] = df_pivot.apply(
            lambda row: format_best_second(row, method_cols, criteria[metric]), axis=1
        )


    # Reset the index to include 'model' and 'user_group' in the DataFrame
    df_reset = df_pivot.reset_index()

    # Export to LaTeX while keeping multicolumn and multirow formatting
    latex = df_reset.to_latex(
        escape=False,  # To allow LaTeX formatting
        index=False,   # Don't write the default DataFrame index
        multicolumn=True,
        multirow=True,
        column_format="llcccccccccccc",  # Adjust for the number of columns
        header=True
    )

    # Modify LaTeX for custom formatting
    latex = latex.replace(
        "\\toprule", 
        "\\toprule\n\\multicolumn{2}{c}{} & \\multicolumn{4}{c}{\\textbf{ndcg}} & \\multicolumn{4}{c}{\\textbf{arp}} & \\multicolumn{4}{c}{\\textbf{poplift}} \\\\ \\cmidrule(lr){3-6} \\cmidrule(lr){7-10} \\cmidrule(lr){11-14}"
    )
    latex = latex.replace("\\midrule", "\\hline")
    latex = latex.replace("\\bottomrule", "\\hline")

    # Save

    with open(f"{BASE_DIR}/{dataset}_dataset/{dataset}_eval.tex", "w") as f:
        f.write(latex)  

    


/var/folders/9q/p0b6nwnd5071040x3849mdvc0000gn/T/ipykernel_351/3468836179.py:16: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  formatted[sorted_indices[0]] = f"\\textbf{{{formatted[sorted_indices[0]]}}}"  # Bold
/var/folders/9q/p0b6nwnd5071040x3849mdvc0000gn/T/ipykernel_351/3468836179.py:16: FutureWarning: Series.__setitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To set a value by position, use `ser.iloc[pos] = value`
  formatted[sorted_indices[0]] = f"\\textbf{{{formatted[sorted_indices[0]]}}}"  # Bold
/var/folders/9q/p0b6nwnd5071040x3849mdvc0000gn/T/ipykernel_351/3468836179.py:18: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will